In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
from utils.depth_operations_functionnal import get_coords_2d, get_rot_mat, \
    repeat_const
import tensorflow as tf
from tensorflow import keras as ks, float32
from keras import backend as K

@tf.function
def depth2disp_former(depth, rot, trans, camera):
    """ Converts a depth map into a disparity map according to given camera motion and specifications """

    with tf.compat.v1.name_scope("depth2disp"):
        b, h, w = depth.get_shape().as_list()[0:3]

        coords2d, _ = get_coords_2d(depth, camera)

        depth = tf.reshape(depth, [b, h * w, 1, 1])
        coords2d = tf.reshape(coords2d, [b, h * w, 3, 1])
        rot_mat = tf.expand_dims(get_rot_mat(rot), axis=1)
        t = tf.reshape(trans, [b, 1, 3, 1])
        f_vec = tf.reshape(tf.concat([camera["f"], tf.ones([b, 1])], axis=1),
                           [b, 1, 3, 1])

        rot_coords = rot_mat @ coords2d
        alpha = rot_coords[:, :, -1:, :]
        proj_coords = rot_coords * f_vec / alpha
        scaled_t = t * f_vec

        delta_x = scaled_t[:, :, 0, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                                :, 0, 0]
        delta_y = scaled_t[:, :, 1, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                                :, 1, 0]

        sqrt_value = tf.reshape(tf.sqrt(delta_x ** 2 + delta_y ** 2),
                                [b, h * w, 1, 1])

        disp = sqrt_value / (depth * alpha + scaled_t[:, :, -1:, :])

        return tf.reshape(disp, [b, h, w, 1])

@tf.function
def depth2disp(depth, rot, trans, camera):
    """ Converts a depth map into a disparity map according to given camera motion and specifications """

    b, h, w = depth.get_shape().as_list()[0:3]

    coords2d, _ = get_coords_2d(depth, camera)

    depth = ks.layers.Reshape((h * w, 1, 1,), )(depth)
    coords2d = ks.layers.Reshape((h * w, 3, 1,), )(coords2d)

    rot_mat = get_rot_mat(rot)
    rot_mat = ks.layers.Reshape((1, rot_mat.shape[1], rot_mat.shape[2],), )(rot_mat)
    t = ks.layers.Reshape((1, 3, 1,), )(trans)

    myconst = tf.convert_to_tensor(np.ones((1, 1)).astype('float32'))
    ones_ = tf.keras.layers.Lambda(lambda x: repeat_const(x, myconst))(camera["f"])
    f_vec = ks.layers.Concatenate(axis=1)([camera["f"], ones_])
    f_vec =  ks.layers.Reshape((1, 3, 1,),)(f_vec)

    rot_coords = rot_mat @ coords2d
    alpha = rot_coords[:, :, -1:, :]
    proj_coords = rot_coords * f_vec / alpha
    scaled_t = t * f_vec

    delta_x = scaled_t[:, :, 0, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                            :, 0, 0]
    delta_y = scaled_t[:, :, 1, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                            :, 1, 0]
    sqrt_value = tf.sqrt(delta_x ** 2 + delta_y ** 2)
    sqrt_value = ks.layers.Reshape((h * w, 1, 1,), ) (sqrt_value)

    disp = sqrt_value / (depth * alpha + scaled_t[:, :, -1:, :])

    disp = ks.layers.Reshape((h, w, 1,), ) (disp)

    return disp


disp = ks.Input(shape=(24, 24, 1,), dtype=float32)  # data image
camera = {"f": ks.Input(shape=(2,), dtype=float32),
          "c": ks.Input(shape=(2,), dtype=float32)}

rot_input = ks.Input(shape=(4), dtype=float32,
                     name="rot_input")  # data camera displacement
trans_input = ks.Input(shape=(3), dtype=float32,
                       name="trans_input")  # data camera displacement
# output: TensorShape([1, 48, 48, 1])

b=1
disp_test_1 = tf.random.uniform([b] + disp.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}
rot_test_1 = tf.random.uniform([b] + rot_input.shape[1:])
trans_test_1 = tf.random.uniform([b] + trans_input.shape[1:])

out_put_test_1_former = depth2disp_former(disp_test_1, rot_test_1, trans_test_1, camera_test_1)
print("out_put_test_1_former", out_put_test_1_former.shape, disp_test_1.shape)
print("---")

depth_curr_l_function = lambda x: depth2disp(x[0], x[1], x[2], x[3])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((disp,rot_input, trans_input,camera))

model_full = ks.Model(inputs=[disp, camera, rot_input, trans_input],
                      outputs=depth_curr_l)

out_put_test_1 = model_full((disp_test_1, camera_test_1, rot_test_1, trans_test_1))

print("disp_test_1", disp_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, disp_test_1.shape)

print(out_put_test_1[0][0][0:3])
print(out_put_test_1_former[0][0][0:3])
print()


b=50
disp_test_1 = tf.random.uniform([b] + disp.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}
rot_test_1 = tf.random.uniform([b] + rot_input.shape[1:])
trans_test_1 = tf.random.uniform([b] + trans_input.shape[1:])

out_put_test_1_former = depth2disp_former(disp_test_1, rot_test_1, trans_test_1, camera_test_1)
print("out_put_test_1_former", out_put_test_1_former.shape, disp_test_1.shape)
print("---")

depth_curr_l_function = lambda x: depth2disp(x[0], x[1], x[2], x[3])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((disp,rot_input, trans_input,camera))

model_full = ks.Model(inputs=[disp, camera, rot_input, trans_input],
                      outputs=depth_curr_l)

out_put_test_1 = model_full((disp_test_1, camera_test_1, rot_test_1, trans_test_1))

print("disp_test_1", disp_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, disp_test_1.shape)

print(out_put_test_1[0][0][0:3])
print(out_put_test_1_former[0][0][0:3])
print()


out_put_test_1_former (1, 24, 24, 1) (1, 24, 24, 1)
---
disp_test_1 (1, 24, 24, 1)
out_put_test_1 (1, 24, 24, 1) (1, 24, 24, 1)
tf.Tensor(
[[ 0.0046895 ]
 [-0.00181194]
 [-0.00105495]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 0.0046895 ]
 [-0.00181194]
 [-0.00105495]], shape=(3, 1), dtype=float32)

out_put_test_1_former (50, 24, 24, 1) (50, 24, 24, 1)
---
disp_test_1 (50, 24, 24, 1)
out_put_test_1 (50, 24, 24, 1) (50, 24, 24, 1)
tf.Tensor(
[[4.4485908 ]
 [1.798833  ]
 [0.51340675]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[4.4485908 ]
 [1.798833  ]
 [0.51340675]], shape=(3, 1), dtype=float32)

